# Task 2: Mining Cancer Feature Patterns

## Objective
Transform the numerical Breast Cancer Wisconsin dataset into categorical sequences and use sequential pattern mining to find patterns that distinguish malignant from benign cases.

## Overview
This notebook implements:
1. Data preprocessing and feature discretization
2. Z-score based feature ranking and sequence generation
3. Sequential pattern mining using PrefixSpan algorithm
4. Sensitivity analysis across different binning strategies


## 1. Setup and Data Loading


In [1]:
# Install required library
!pip install prefixspan-py


ERROR: Could not find a version that satisfies the requirement prefixspan-py (from versions: none)
ERROR: No matching distribution found for prefixspan-py


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
import prefixspan
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")


Libraries imported successfully!


In [3]:
# Load the dataset
df = pd.read_csv('./datasets/Cancer_Data.csv')
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
df.head()


Dataset shape: (569, 33)
Columns: ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32']

First few rows:


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
# Data cleaning
# Drop the id and Unnamed: 32 columns
df_clean = df.drop(['id', 'Unnamed: 32'], axis=1, errors='ignore')
print(f"Shape after dropping columns: {df_clean.shape}")

# Separate features and target
X = df_clean.drop('diagnosis', axis=1)
y = df_clean['diagnosis']

# Encode diagnosis: M (Malignant) = 1, B (Benign) = 0
y_encoded = (y == 'M').astype(int)

print(f"Features shape: {X.shape}")
print(f"Target distribution:")
print(f"  Benign (0): {(y_encoded == 0).sum()}")
print(f"  Malignant (1): {(y_encoded == 1).sum()}")
print(f"\nFeature columns: {list(X.columns)}")


Shape after dropping columns: (569, 31)
Features shape: (569, 30)
Target distribution:
  Benign (0): 357
  Malignant (1): 212

Feature columns: ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']


## 2. Feature Transformation Pipeline


In [5]:
def transform_features_to_sequences(X, y, k=10, n_bins=3, strategy='uniform'):
    """
    Transform numerical features into categorical sequences for pattern mining.
    
    Parameters:
    -----------
    X : pandas.DataFrame
        Feature matrix
    y : pandas.Series or numpy.array
        Target labels (0 for benign, 1 for malignant)
    k : int
        Number of top features to select for each patient
    n_bins : int
        Number of bins for discretization
    strategy : str
        Discretization strategy ('uniform', 'quantile', 'kmeans')
    
    Returns:
    --------
    malignant_sequences : list
        List of sequences for malignant cases
    benign_sequences : list
        List of sequences for benign cases
    """
    
    # Step 1: Discretize Features
    print(f"Discretizing features using {strategy} strategy with {n_bins} bins...")
    
    # Initialize discretizer
    discretizer = KBinsDiscretizer(
        n_bins=n_bins, 
        encode='ordinal', 
        strategy=strategy
    )
    
    # Fit and transform features
    X_discretized = discretizer.fit_transform(X)
    
    # Create descriptive labels for discretized features
    bin_labels = ['low', 'medium', 'high']
    feature_labels = []
    
    for i, col in enumerate(X.columns):
        for j, label in enumerate(bin_labels):
            feature_labels.append(f"{col}_{label}")
    
    print(f"Created {len(feature_labels)} discretized feature labels")
    
    # Step 2: Calculate Z-scores and generate sequences
    print(f"Calculating z-scores and generating sequences with top-{k} features...")
    
    malignant_sequences = []
    benign_sequences = []
    
    for idx in range(len(X)):
        # Calculate z-scores for this patient
        patient_values = X.iloc[idx].values
        feature_means = X.mean().values
        feature_stds = X.std().values
        
        # Avoid division by zero
        feature_stds = np.where(feature_stds == 0, 1, feature_stds)
        
        z_scores = np.abs((patient_values - feature_means) / feature_stds)
        
        # Get top-k features with highest absolute z-scores
        top_k_indices = np.argsort(z_scores)[-k:][::-1]  # Sort descending
        
        # Create sequence using discretized labels
        sequence = []
        for feat_idx in top_k_indices:
            bin_idx = int(X_discretized[idx, feat_idx])
            feature_name = X.columns[feat_idx]
            bin_label = bin_labels[bin_idx]
            sequence.append(f"{feature_name}_{bin_label}")
        
        # Add to appropriate list based on diagnosis
        if y.iloc[idx] == 1:  # Malignant
            malignant_sequences.append(sequence)
        else:  # Benign
            benign_sequences.append(sequence)
    
    print(f"Generated {len(malignant_sequences)} malignant sequences")
    print(f"Generated {len(benign_sequences)} benign sequences")
    
    return malignant_sequences, benign_sequences


## 3. Sequential Pattern Mining and Sensitivity Analysis


In [6]:
# Define parameters
k = 10  # Number of top features to select
n_bins = 3  # Number of bins for discretization
min_support = 0.2  # Minimum support threshold (20%)
top_patterns = 10  # Number of top patterns to extract

# Strategies to test
strategies = ['uniform', 'quantile', 'kmeans']

print("Starting sensitivity analysis across different binning strategies...")
print("=" * 80)


Starting sensitivity analysis across different binning strategies...


In [7]:
# Main sensitivity analysis loop
results = {}

for strategy in strategies:
    print(f"\n{'='*20} STRATEGY: {strategy.upper()} {'='*20}")
    
    # Transform features to sequences
    malignant_seqs, benign_seqs = transform_features_to_sequences(
        X, y_encoded, k=k, n_bins=n_bins, strategy=strategy
    )
    
    # Apply PrefixSpan to malignant sequences
    print(f"\nMining patterns in MALIGNANT sequences...")
    print(f"Total malignant sequences: {len(malignant_seqs)}")
    
    if len(malignant_seqs) > 0:
        malignant_patterns = prefixspan.PrefixSpan(malignant_seqs).frequent(
            min_support * len(malignant_seqs)
        )
        
        # Get top patterns for malignant
        malignant_top = sorted(malignant_patterns, key=lambda x: x[0], reverse=True)[:top_patterns]
        
        print(f"\nTop {len(malignant_top)} MALIGNANT patterns:")
        for i, (support, pattern) in enumerate(malignant_top, 1):
            print(f"  {i:2d}. Support: {support:3d} | Pattern: {' -> '.join(pattern)}")
    else:
        malignant_top = []
        print("No malignant sequences found!")
    
    # Apply PrefixSpan to benign sequences
    print(f"\nMining patterns in BENIGN sequences...")
    print(f"Total benign sequences: {len(benign_seqs)}")
    
    if len(benign_seqs) > 0:
        benign_patterns = prefixspan.PrefixSpan(benign_seqs).frequent(
            min_support * len(benign_seqs)
        )
        
        # Get top patterns for benign
        benign_top = sorted(benign_patterns, key=lambda x: x[0], reverse=True)[:top_patterns]
        
        print(f"\nTop {len(benign_top)} BENIGN patterns:")
        for i, (support, pattern) in enumerate(benign_top, 1):
            print(f"  {i:2d}. Support: {support:3d} | Pattern: {' -> '.join(pattern)}")
    else:
        benign_top = []
        print("No benign sequences found!")
    
    # Store results
    results[strategy] = {
        'malignant_patterns': malignant_top,
        'benign_patterns': benign_top,
        'malignant_sequences': malignant_seqs,
        'benign_sequences': benign_seqs
    }
    
    print(f"\n{'='*60}")



==================== STRATEGY: UNIFORM ====================
Discretizing features using uniform strategy with 3 bins...
Created 90 discretized feature labels
Calculating z-scores and generating sequences with top-10 features...


Generated 212 malignant sequences
Generated 357 benign sequences

Mining patterns in MALIGNANT sequences...
Total malignant sequences: 212

Top 10 MALIGNANT patterns:
   1. Support:  85 | Pattern: perimeter_mean_medium
   2. Support:  83 | Pattern: perimeter_worst_medium
   3. Support:  82 | Pattern: radius_worst_medium
   4. Support:  81 | Pattern: radius_mean_medium
   5. Support:  77 | Pattern: area_mean_medium
   6. Support:  72 | Pattern: concave points_mean_medium
   7. Support:  64 | Pattern: concave points_worst_high
   8. Support:  58 | Pattern: concavity_mean_medium
   9. Support:  57 | Pattern: area_worst_medium
  10. Support:  56 | Pattern: radius_mean_medium -> perimeter_mean_medium

Mining patterns in BENIGN sequences...
Total benign sequences: 357

Top 10 BENIGN patterns:
   1. Support: 142 | Pattern: concave points_worst_low
   2. Support: 134 | Pattern: concavity_worst_low
   3. Support: 124 | Pattern: compactness_mean_low
   4. Support: 120 | Pattern: texture_worst_lo

Generated 212 malignant sequences
Generated 357 benign sequences

Mining patterns in MALIGNANT sequences...
Total malignant sequences: 212

Top 10 MALIGNANT patterns:
   1. Support: 104 | Pattern: perimeter_mean_high
   2. Support: 100 | Pattern: perimeter_worst_high
   3. Support: 100 | Pattern: radius_mean_high
   4. Support: 100 | Pattern: radius_worst_high
   5. Support:  98 | Pattern: concave points_worst_high
   6. Support:  93 | Pattern: concave points_mean_high
   7. Support:  88 | Pattern: area_mean_high
   8. Support:  77 | Pattern: area_worst_high
   9. Support:  74 | Pattern: concavity_mean_high
  10. Support:  72 | Pattern: compactness_worst_high

Mining patterns in BENIGN sequences...
Total benign sequences: 357

Top 10 BENIGN patterns:
   1. Support: 136 | Pattern: concave points_worst_low
   2. Support: 125 | Pattern: concavity_worst_low
   3. Support: 121 | Pattern: compactness_mean_low
   4. Support: 120 | Pattern: texture_worst_low
   5. Support: 117 | Pattern: perim

Created 90 discretized feature labels
Calculating z-scores and generating sequences with top-10 features...


Generated 212 malignant sequences
Generated 357 benign sequences

Mining patterns in MALIGNANT sequences...
Total malignant sequences: 212

Top 10 MALIGNANT patterns:
   1. Support:  89 | Pattern: radius_mean_high
   2. Support:  87 | Pattern: perimeter_mean_high
   3. Support:  86 | Pattern: concave points_worst_high
   4. Support:  77 | Pattern: radius_worst_high
   5. Support:  75 | Pattern: perimeter_worst_high
   6. Support:  73 | Pattern: area_mean_medium
   7. Support:  57 | Pattern: area_worst_medium
   8. Support:  57 | Pattern: radius_mean_high -> perimeter_mean_high
   9. Support:  57 | Pattern: texture_worst_high
  10. Support:  56 | Pattern: radius_se_medium

Mining patterns in BENIGN sequences...
Total benign sequences: 357

Top 10 BENIGN patterns:
   1. Support: 142 | Pattern: concave points_worst_low
   2. Support: 130 | Pattern: concavity_worst_low
   3. Support: 124 | Pattern: compactness_mean_low
   4. Support: 120 | Pattern: texture_worst_low
   5. Support: 120 | Pa

## 4. Interpretation and Analysis


### How to Interpret the Results

The sequential pattern mining results show frequent patterns in the discretized feature sequences for malignant and benign cases. Here's how to interpret them:

1. **Support Count**: The number of sequences that contain this pattern
2. **Pattern**: The sequence of discretized features (e.g., 'radius_mean_high -> texture_mean_medium')
3. **Discriminating Patterns**: Look for patterns that are frequent in one group but rare/absent in the other

### Key Questions to Answer:
- Which patterns are unique to malignant cases?
- Which patterns are unique to benign cases?
- How do different binning strategies affect the discovered patterns?
- Are there consistent patterns across different strategies?


In [8]:
# Function to find discriminating patterns
def find_discriminating_patterns(malignant_patterns, benign_patterns, min_difference=5):
    """
    Find patterns that are significantly more frequent in one group than the other.
    """
    # Convert patterns to dictionaries for easy lookup
    mal_dict = {tuple(pattern): support for support, pattern in malignant_patterns}
    ben_dict = {tuple(pattern): support for support, pattern in benign_patterns}
    
    discriminating = {
        'malignant_unique': [],
        'benign_unique': [],
        'malignant_dominant': [],
        'benign_dominant': []
    }
    
    # Find patterns unique to malignant
    for pattern, support in mal_dict.items():
        if pattern not in ben_dict:
            discriminating['malignant_unique'].append((support, pattern))
        elif support - ben_dict[pattern] >= min_difference:
            discriminating['malignant_dominant'].append((support, ben_dict[pattern], pattern))
    
    # Find patterns unique to benign
    for pattern, support in ben_dict.items():
        if pattern not in mal_dict:
            discriminating['benign_unique'].append((support, pattern))
        elif support - mal_dict[pattern] >= min_difference:
            discriminating['benign_dominant'].append((support, mal_dict[pattern], pattern))
    
    return discriminating


In [9]:
# Analyze discriminating patterns for each strategy
print("DISCRIMINATING PATTERN ANALYSIS")
print("=" * 50)

for strategy in strategies:
    print(f"\n{strategy.upper()} STRATEGY:")
    print("-" * 30)
    
    mal_patterns = results[strategy]['malignant_patterns']
    ben_patterns = results[strategy]['benign_patterns']
    
    discriminating = find_discriminating_patterns(mal_patterns, ben_patterns)
    
    print(f"\nPatterns UNIQUE to MALIGNANT cases:")
    if discriminating['malignant_unique']:
        for support, pattern in discriminating['malignant_unique'][:5]:  # Show top 5
            print(f"  Support: {support:3d} | Pattern: {' -> '.join(pattern)}")
    else:
        print("  None found")
    
    print(f"\nPatterns UNIQUE to BENIGN cases:")
    if discriminating['benign_unique']:
        for support, pattern in discriminating['benign_unique'][:5]:  # Show top 5
            print(f"  Support: {support:3d} | Pattern: {' -> '.join(pattern)}")
    else:
        print("  None found")
    
    print(f"\nPatterns DOMINANT in MALIGNANT cases:")
    if discriminating['malignant_dominant']:
        for mal_sup, ben_sup, pattern in discriminating['malignant_dominant'][:3]:
            print(f"  Malignant: {mal_sup:3d}, Benign: {ben_sup:3d} | Pattern: {' -> '.join(pattern)}")
    else:
        print("  None found")
    
    print(f"\nPatterns DOMINANT in BENIGN cases:")
    if discriminating['benign_dominant']:
        for ben_sup, mal_sup, pattern in discriminating['benign_dominant'][:3]:
            print(f"  Benign: {ben_sup:3d}, Malignant: {mal_sup:3d} | Pattern: {' -> '.join(pattern)}")
    else:
        print("  None found")
    
    print("\n" + "="*50)


DISCRIMINATING PATTERN ANALYSIS

UNIFORM STRATEGY:
------------------------------

Patterns UNIQUE to MALIGNANT cases:
  Support:  85 | Pattern: perimeter_mean_medium
  Support:  83 | Pattern: perimeter_worst_medium
  Support:  82 | Pattern: radius_worst_medium
  Support:  81 | Pattern: radius_mean_medium
  Support:  77 | Pattern: area_mean_medium

Patterns UNIQUE to BENIGN cases:
  Support: 142 | Pattern: concave points_worst_low
  Support: 134 | Pattern: concavity_worst_low
  Support: 124 | Pattern: compactness_mean_low
  Support: 120 | Pattern: texture_worst_low
  Support: 120 | Pattern: perimeter_mean_low

Patterns DOMINANT in MALIGNANT cases:
  None found

Patterns DOMINANT in BENIGN cases:
  None found


QUANTILE STRATEGY:
------------------------------

Patterns UNIQUE to MALIGNANT cases:
  Support: 104 | Pattern: perimeter_mean_high
  Support: 100 | Pattern: perimeter_worst_high
  Support: 100 | Pattern: radius_mean_high
  Support: 100 | Pattern: radius_worst_high
  Support:  9

## 5. Summary and Conclusions


In [10]:
# Summary statistics
print("SUMMARY STATISTICS")
print("=" * 30)

for strategy in strategies:
    mal_count = len(results[strategy]['malignant_patterns'])
    ben_count = len(results[strategy]['benign_patterns'])
    mal_seqs = len(results[strategy]['malignant_sequences'])
    ben_seqs = len(results[strategy]['benign_sequences'])
    
    print(f"\n{strategy.upper()} Strategy:")
    print(f"  Malignant sequences: {mal_seqs}")
    print(f"  Benign sequences: {ben_seqs}")
    print(f"  Malignant patterns found: {mal_count}")
    print(f"  Benign patterns found: {ben_count}")
    print(f"  Total patterns: {mal_count + ben_count}")


SUMMARY STATISTICS

UNIFORM Strategy:
  Malignant sequences: 212
  Benign sequences: 357
  Malignant patterns found: 10
  Benign patterns found: 10
  Total patterns: 20

QUANTILE Strategy:
  Malignant sequences: 212
  Benign sequences: 357
  Malignant patterns found: 10
  Benign patterns found: 10
  Total patterns: 20

KMEANS Strategy:
  Malignant sequences: 212
  Benign sequences: 357
  Malignant patterns found: 10
  Benign patterns found: 10
  Total patterns: 20


### Key Findings and Next Steps

1. **Pattern Discovery**: The sequential pattern mining has identified frequent patterns in both malignant and benign cases.

2. **Strategy Comparison**: Different binning strategies (uniform, quantile, kmeans) may reveal different aspects of the data:
   - **Uniform**: Equal-width bins may capture absolute feature values
   - **Quantile**: Equal-frequency bins may capture relative rankings
   - **K-means**: Data-driven bins may capture natural clusters

3. **Discriminating Patterns**: Look for patterns that are:
   - Unique to one class (highly discriminating)
   - Significantly more frequent in one class
   - Consistent across different strategies

4. **Clinical Interpretation**: The discovered patterns can be interpreted in terms of:
   - Feature combinations that indicate malignancy risk
   - Sequential relationships between different cancer characteristics
   - Potential biomarkers for early detection

5. **Further Analysis**: Consider:
   - Adjusting the minimum support threshold
   - Varying the number of top features (k)
   - Exploring different discretization parameters
   - Validating patterns on independent datasets
